This notebook is used for exploration of the ICD API.

In [ ]:
import httpx
import os

In [ ]:
# --- ICD API Config ---
TOKEN_URL = "https://icdaccessmanagement.who.int/connect/token"
ICD_SEARCH_URL = "https://id.who.int/icd/release/11/2022-02/mms/search"

CLIENT_ID = os.environ.get("ICD_CLIENT_ID", "")
CLIENT_SECRET = os.environ.get("ICD_CLIENT_SECRET", "")
SCOPE = "icdapi_access"
GRANT_TYPE = "client_credentials"

payload = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "scope": SCOPE,
    "grant_type": GRANT_TYPE,
}


def get_token(payload: dict, token_url: str = TOKEN_URL) -> str:
    """
    Authenticate with the WHO ICD API and return an access token.
    """

    with httpx.Client(verify=True) as client:
        response = client.post(token_url, data=payload)
        response.raise_for_status()
        token = response.json().get("access_token")
        return token

In [ ]:
def lookup_icd11(term: str, token: str) -> dict:
    """
    Search for ICD-11 codes using a search term and return the results.
    """
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json",
        "Accept-Language": "en",
        "API-Version": "v2",
    }

    params = {
        "q": term,
        "useFlexisearch": "true",
    }

    with httpx.Client(verify=False) as client:
        response = client.get(ICD_SEARCH_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        candidates = [
            {
                "code": candidate.get("theCode"),
                "score": candidate.get("score"),
                "title": candidate.get("title"),
            } for candidate in data.get("destinationEntities", [])
        ]

        # filter for score  > 0.3 # TODO: maybe make this a parameter
        candidates = [c for c in candidates if c["score"] > 0.3]
        # sort by score descending
        candidates.sort(key=lambda x: x["score"], reverse=True)

        return {
            "term": term,
            "candidates": candidates,
        }

Score of >0.3 seems to be a good cut-off for the model to be able to predict the correct class.

In [ ]:
lookup_icd11("metastasis", get_token(payload=payload))